In [3]:
# Imports
from buildingmotif import BuildingMOTIF
from rdflib import Namespace
from buildingmotif.dataclasses import Model
import csv


# create the namespace
BLDG = Namespace('urn:bldg/')
bm = BuildingMOTIF("sqlite://") # in-memory instance

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="Test Model") 

# Should add volttron configs to triple information

In [53]:
# funcs
def read_csv(path):
    line = 0
    contents = []
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            contents.append(row)
        return contents


def create_relation(contents):
    g.add(())

def instantiate_equips():
    pass

def add_points():
    pass


In [57]:
filename  = '/Users/lazlopaul/Desktop/VolttronSemantics/files/entity_relations.csv'
contents = read_csv(filename)
#print(contents)
for item in contents:
    objects = item.get('objectURI')
    for object
    print(object)

Zone1
Zone1
Zone1
Zone1
Zone1
Zone1
Zone1
Boiler1,Fan1,HeatEx1
Boiler2,Fan2,HeatEx2
HeatEx1
Fan1
HeatEx2
Fan2
Office,AdminSpace
Office,AdminSpace


In [ ]:
#Instantiate Every thing in table (URI and Type)
#Then create relations
#Then create points
#Then add the topic names for points to them